In [7]:
import os
import matplotlib.pyplot as plt
from dataset import dataset
from edsr import edsr
from train import EdsrTrainer
from common import resolve
from common import resolve_single
from common import evaluate

In [8]:
# set hp depth and scale and get device to clear memory later

'''
author's settled on 32 res blocks and 256 filters and a scaling
multiple for the residual blocks of 0.1
'''
depth = 16
scale = 4
features = 64

In [9]:
# set directory for saving model weights 

weights_dir = './weights/edsr/'
weights_file = os.path.join(weights_dir, 'weightsB{}F{}.h5'.format(depth, features))
os.makedirs(weights_dir, exist_ok=True)

In [10]:
# create our training and validation datasets

train_ds = dataset(batch_size=16, random_transform=True, subset='train')
valid_ds = dataset(batch_size=1, repeat_count=1, random_transform=False, subset='valid')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

In [12]:
# initialize model and get weights from model trained by Krasserm

model=edsr(scale=scale, num_filters=features, num_res_blocks=depth, res_block_scaling=0.1)
model.load_weights('weights/edsr/weights.h5')     

In [14]:
# initialize the trainer

trainer = EdsrTrainer(model=model, loss='MAE', checkpoint_dir=f'.ckpt/edsr-{depth}-x{scale}')

In [15]:
# train!

trainer.train(train_ds, valid_ds.take(10), steps=60000, evaluate_every=100, save_best_only=True)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
100/60000: loss = 11.143, PSNR = 22.346092
200/60000: loss = 9.839, PSNR = 22.790030
300/60000: loss = 9.345, PSNR = 22.999319
400/60000: loss = 9.099, PSNR = 23.133902
500/60000: loss = 8.943, PSNR = 23.213047
600/60000: loss = 8.812, PSNR = 23.326813
700/60000: loss = 8.692, PSNR = 23.372448
800/60000: loss = 8.631, PSNR = 23.434826
900/60000: loss = 8.577, PSNR = 23.472525
1000/6000

KeyboardInterrupt: 

In [8]:
# restore the weights from the best checkpoint

trainer.restore()

Model restored from checkpoint at step 59400.


In [9]:
# validate results

psnrv = trainer.evaluate(valid_ds)
print(f'PSNR = {psnrv.numpy():3f}')

PSNR = 33.202381


In [11]:
# save weights

trainer.model.save_weights(weights_file)